In [14]:
import numpy as np
import re
import csv
import os
from PIL import Image
import PIL
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing import image
#from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf
import keras


In [15]:
#for i in range (len(imageset)):
    #imageset[i] = os.path.join(directory, imageset[i])
#PIL.Image.open(imageset[i])

In [16]:
#images in the datatset are separate into these folders.
folders = ["3DPeS/archive", "CAVIAR4REID/archive", "CUHK/archive", "GRID/archive", "i-LID/archive", "MIT/archive", "PRID/archive", "SARC3D/archive", "TownCentre/archive", "VIPeR/archive"]
#Creating lists for the image set and label set.
fullimageset = []
fulllabelset = []

#Iterating over the folders to add images and labels to their lists.
for directory in folders:
    #Finding and reading the label file in the folder.
    #Sorting numerically.
    labelfile = os.path.join(directory, "Label.txt")
    file = open (labelfile)
    lines = file.readlines()
    lines = sorted(lines, key=lambda s: int(re.findall(r'\d+', s)[0]))

    labels = np.ndarray(shape = len(lines))
    labelnames = np.empty(shape = len(lines))
    
    #Substrings within the label files used for each age group.
    sub1 = "personalLess15"
    sub2 = "personalLess30"
    sub3 = "personalLess45"
    sub4 = "personalLess60"
    sub5 = "personalLarger60"

    #Store the corresponding age group in labels.
    # -1 if age group is not specified.
    for i in range (len(lines)):
        if sub1 in lines[i]:
            labels[i] = 0
        elif sub2 in lines[i]:
            labels[i] = 1
        elif sub3 in lines[i]:
            labels[i] = 2
        elif sub4 in lines[i]:
            labels[i] = 3
        elif sub5 in lines[i]:
            labels[i] = 4
        else:
            labels[i] = -1

    #Obtaining the image name of each label, present at the start of each line in the label file.
    #Removing ".png" from the end of the name for CUHK folder.
    with open(labelfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=' ')
        line_count = 0
        for row in csv_reader:
            if directory == "CUHK/archive":
                row[0] = row[0][:4]
            labelnames[line_count] = row[0]
            line_count += 1
    labelnames.sort()

    
    # Creating a dictionary where the keys are the image names and values are label values.
    dictionary = {}

    for i in range (len(labels)):
        dictionary[int(labelnames[i])] = int(labels[i])


    #Obtaining all images in the folder.
    #Removing the last file in the list, Labels.txt.
    #Second list used to store only images with labels value != -1
    imageset = os.listdir(directory)
    imageset.pop(-1)
    newimageset = []
    labelset = []
    
    #Each directory has a different numbering format
    #Here the name of each image is being obtained
    #If an age label exists, the image and label are added to the set
    if directory == "TownCentre/archive" or directory == "3DPeS/archive":
        for i in range (len(imageset)):
            index = imageset[i][:3]
            if index[2] == "_":
                index = index[:2]
            elif index[1] == "_":
                index = index[:1]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    elif directory == "SARC3D/archive" or directory == "VIPeR/archive":
        for i in range (len(imageset)):
            index = imageset[i][:3]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
                #labelset[i] = dictionary.get(int(index))
    elif directory == "MIT/archive":
        for i in range (len(imageset)):
            index = imageset[i][:5]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    else:

        for i in range (len(imageset)):
            index = imageset[i][:4]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    
    #At the end of the loop, elements are appended to the full image and label sets
    for i in range (len(newimageset)):
        imageset[i] = os.path.join(directory, imageset[i])
        fullimageset.append(newimageset[i])
        fulllabelset.append(labelset[i])





In [17]:
#Showing the number of images for each age range
fifteen = 0
thirty = 0
fortyfive = 0
sixty = 0
sixtyplus = 0

for i in fulllabelset:
    if i == 0: fifteen +=1
    if i == 1: thirty +=1
    if i == 2: fortyfive +=1
    if i == 3: sixty +=1
    if i == 4: sixtyplus +=1

print(fifteen)
print(thirty)
print(fortyfive)
print(sixty)
print(sixtyplus)

170
9443
6246
1943
1170


In [18]:
#Splitting the sets into training and test sets
X_train, X_test, y_train, y_test = train_test_split(fullimageset, fulllabelset, test_size=0.33, random_state=42)

#Converting the labels to strings to convert to pandas dataframe
y_trains = []
y_tests = []


for i in range (len(y_train)):
        if int(y_train[i]) == 0:
            y_trains.append(sub1)
        elif int(y_train[i]) == 1:
            y_trains.append(sub2)
        elif int(y_train[i]) == 2:
            y_trains.append(sub3)
        elif int(y_train[i]) == 3:
            y_trains.append(sub4)
        elif int(y_train[i]) == 4:
            y_trains.append(sub5)
        else:
            print(y_train[i])

            
for i in range (len(y_test)):
        if int(y_test[i]) == 0:
            y_tests.append(sub1)
        elif int(y_test[i]) == 1:
            y_tests.append(sub2)
        elif int(y_test[i]) == 2:
            y_tests.append(sub3)
        elif int(y_test[i]) == 3:
            y_tests.append(sub4)
        elif int(y_test[i]) == 4:
            y_tests.append(sub5)


            


In [19]:
#Convert datasets to pandas dataframe
traindf = pd.DataFrame({"Images":X_train, "Labels":y_trains})
testdf = pd.DataFrame({"Images":X_test, "Labels":y_tests})

In [20]:
print(traindf)
print(testdf)

                                      Images            Labels
0       3DPeS/archive\6_575_FRAME_75_RGB.bmp    personalLess30
1            TownCentre/archive\196_4298.jpg    personalLess45
2                      CUHK/archive\1114.png    personalLess45
3                      CUHK/archive\2071.png    personalLess30
4            MIT/archive\00218_male_fore.jpg    personalLess45
...                                      ...               ...
12706        TownCentre/archive\118_2050.jpg    personalLess45
12707        TownCentre/archive\138_2409.jpg  personalLarger60
12708                  CUHK/archive\3160.png    personalLess30
12709  3DPeS/archive\6_481_FRAME_743_RGB.bmp    personalLess30
12710          TownCentre/archive\43_983.jpg    personalLess45

[12711 rows x 2 columns]
                                              Images            Labels
0                         VIPeR/archive\503_90_2.bmp    personalLess30
1                            PRID/archive\1265_b.png    personalLess45
2    

In [21]:
#delete
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [22]:
#Converting labels from strings to one hot encoding
labeldic = {"personalLess15":0, "personalLess30": 1, "personalLess45":2, "personalLess60":3, "personalLarger60":4}

intytrains = []
for i in range (len(y_trains)):
    intytrains.append(labeldic.get(y_trains[i]))
labels_train = tf.keras.utils.to_categorical(intytrains)

intytest = []
for i in range (len(y_tests)):
    intytest.append(labeldic.get(y_tests[i]))
labels_test = tf.keras.utils.to_categorical(intytest)

In [23]:
#Defining training and testing generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=traindf, directory=None, x_col="Images", y_col="Labels", target_size=(128, 48), batch_size= 8, class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(dataframe=testdf, directory=None, x_col="Images", y_col="Labels", target_size=(128, 48), batch_size= 8,
                                                           class_mode='categorical')


Found 12711 validated image filenames belonging to 5 classes.
Found 6261 validated image filenames belonging to 5 classes.


In [24]:

ptmodel = tf.keras.applications.resnet50.ResNet50(
                    input_shape=(128,48, 3),
                    include_top=False,
                    weights='imagenet',
                    pooling='avg')


In [25]:
#Defining model and layers
inputs = ptmodel.input
input_shape = (128, 48, 3)
x = tf.keras.layers.Dense(128, activation='relu')(ptmodel.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 48, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 54, 3)   0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 24, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 24, 64)   256         ['conv1_conv[0][0]']       

 conv2_block3_2_conv (Conv2D)   (None, 32, 12, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 12, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 12, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 12, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 6, 128)   147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 6, 128)  512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 6, 128)  0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 6, 512)   66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 3, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 3, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 3, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 3, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 8, 3, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 8, 3, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 8, 3, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 8, 3, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 2, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [28]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [30]:
#Defining training parameters and fitting the model
model.compile(optimizer='adam',loss=weighted_categorical_crossentropy([28,0.5,0.75,2.5,4]),metrics=[tf.keras.metrics.RootMeanSquaredError()])


history = model.fit(train_generator, epochs = 10)#(x= np.array(X_train), y = np.array(labels_train), epochs=25)

Epoch 1/10
1589/1589 [==============================] - 551s 343ms/step - loss: 3.6499 - root_mean_squared_error: 0.4098
Epoch 2/10
1589/1589 [==============================] - 555s 349ms/step - loss: 3.3420 - root_mean_squared_error: 0.4014
Epoch 3/10
1589/1589 [==============================] - 611s 385ms/step - loss: 3.2943 - root_mean_squared_error: 0.3979
Epoch 4/10
1589/1589 [==============================] - 736s 463ms/step - loss: 3.0731 - root_mean_squared_error: 0.3921
Epoch 5/10
1589/1589 [==============================] - 744s 468ms/step - loss: 3.0981 - root_mean_squared_error: 0.3894
Epoch 6/10
1589/1589 [==============================] - 744s 469ms/step - loss: 2.9325 - root_mean_squared_error: 0.3871
Epoch 7/10
1589/1589 [==============================] - 712s 448ms/step - loss: 2.7713 - root_mean_squared_error: 0.3778
Epoch 8/10
1589/1589 [==============================] - 551s 347ms/step - loss: 2.6538 - root_mean_squared_error: 0.3722
Epoch 9/10
1589/1589 [==========

In [31]:
from sklearn.metrics import accuracy_score
#Testing the model
Y_pred = model.predict(test_generator).round()
#Y_pred = np.argmax(Y_pred ,axis =1)


783/783 [==============================] - 50s 63ms/step


In [34]:
print("Accuracy:")

m = tf.keras.utils.to_categorical(test_generator.classes)
acc = accuracy_score(m, Y_pred)
print(acc)

Accuracy:
0.18176010222009265


In [36]:
prediction = tf.keras.metrics.RootMeanSquaredError()
prediction.update_state(m, Y_pred)
prediction.result().numpy()

0.49726933